# Setup Environment 

In [1]:
pip install opencv-python

In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


# Preprocessing

### Augmented Data + Convert RGB to Grayscale + Filter

In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

# Path ke direktori dataset
dataset_dir = '../data/data_original/javaneseScript_byPhiard/train'
output_dir = '../data/data_preprocessing/byPhiard_v1.2'

datagen = ImageDataGenerator(
    rotation_range=19,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Membuat folder baru untuk menyimpan hasil augmentasi
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Roberts Operator
kernel_roberts_x = np.array([[1, 0], [0, -1]])
kernel_roberts_y = np.array([[0, 1], [-1, 0]])

# Loop melalui setiap kelas dalam dataset
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)

    # Membuat folder baru untuk setiap kelas di dalam folder output
    output_class_path = os.path.join(output_dir, class_name)
    if not os.path.exists(output_class_path):
        os.makedirs(output_class_path)

    # Mendapatkan list file gambar di setiap kelas
    image_files = [os.path.join(class_path, file) for file in os.listdir(class_path) if file.endswith(('jpg', 'jpeg', 'png'))]

    # Loop melalui setiap file gambar dan augmentasi datanya
    for img_path in image_files:
        img = image.load_img(img_path, target_size=(224, 224), color_mode="rgb")  # Memuat gambar dengan mode RGB
        x = image.img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Proses augmented images
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=output_class_path,
                                  save_prefix=os.path.splitext(os.path.basename(img_path))[0] + '_aug', save_format='png'):
            
            augmented_img = batch[0]  # Mendapatkan gambar augmented dari batch
            gray_img = cv2.cvtColor(augmented_img, cv2.COLOR_RGB2GRAY).astype(np.uint8)  # Konversi RGB ke Grayscale dan ubah ke uint8
            
            edges_roberts_x = cv2.filter2D(gray_img, cv2.CV_64F, kernel_roberts_x)
            edges_roberts_y = cv2.filter2D(gray_img, cv2.CV_64F, kernel_roberts_y)
            magnitude_roberts = np.sqrt(edges_roberts_x**2 + edges_roberts_y**2)

            # Thresholding for Edge Detection Results
            threshold = 50
            detected_edges_roberts = np.zeros_like(magnitude_roberts)
            detected_edges_roberts[magnitude_roberts > threshold] = 255
            
            # Simpan gambar hasil augmentasi dengan tepi yang terdeteksi
            cv2.imwrite(os.path.join(output_class_path, os.path.splitext(os.path.basename(img_path))[0] + f'_aug_{i}_edges.png'), detected_edges_roberts)
            
            i += 1
            if i >= 250:
                break
        break

print(f"\nAugmentasi Selesai!")


Augmentasi Selesai!


In [8]:
# Dictionary untuk menyimpan jumlah data di setiap kelas
kelas_data_count = {}

# Loop melalui setiap kelas di dalam folder output
for class_name in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_name)
    
    # Menghitung jumlah file gambar di setiap kelas
    num_images = len([file for file in os.listdir(class_path) if file.endswith(('jpg', 'jpeg', 'png'))])
    
    # Menyimpan jumlah data di setiap kelas ke dalam dictionary
    kelas_data_count[class_name] = num_images

# Menampilkan jumlah data di setiap kelas
for class_name, count in kelas_data_count.items():
    print(f"Kelas {class_name}: {count} gambar")

Kelas ba: 495 gambar
Kelas ca: 499 gambar
Kelas da: 495 gambar
Kelas dha: 497 gambar
Kelas ga: 497 gambar
Kelas ha: 497 gambar
Kelas ja: 498 gambar
Kelas ka: 497 gambar
Kelas la: 495 gambar
Kelas ma: 498 gambar
Kelas na: 498 gambar
Kelas nga: 498 gambar
Kelas nya: 495 gambar
Kelas pa: 497 gambar
Kelas ra: 500 gambar
Kelas sa: 498 gambar
Kelas ta: 495 gambar
Kelas tha: 498 gambar
Kelas wa: 499 gambar
Kelas ya: 497 gambar


### Imbalanced Dataset

In [10]:
import random
import string
import shutil

# Path dataset
data = output_dir

# List kelas
kelas_aksara = ['ba', 'ca', 'da', 'dha', 'ga', 'ha', 'ja', 'ka', 'la', 'ma', 'na', 'nga', 'nya', 'pa', 'ra', 'sa', 'ta', 'tha', 'wa', 'ya']

# Target jumlah gambar per kelas
target_jumlah = 500

# Loop untuk setiap kelas
for kelas in kelas_aksara:
    # Path kelas
    kelas_path = os.path.join(data, kelas)
    
    # Jumlah gambar saat ini
    jumlah_gambar = len(os.listdir(kelas_path))
    
    # Selisih gambar yang perlu ditambahkan
    selisih = target_jumlah - jumlah_gambar
    
    # Jika selisih positif, lakukan oversampling
    if selisih > 0:
        # Ambil sampel acak dari gambar yang sudah ada
        gambar_oversampling = random.sample(os.listdir(kelas_path), selisih)
        
        # Copy gambar oversampling ke dalam kelas
        for gambar in gambar_oversampling:
            source_path = os.path.join(kelas_path, gambar)
            
            # Generate sufiks acak
            sufiks_acak = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
            
            # Tentukan path tujuan dengan menambahkan sufiks acak
            target_path = os.path.join(kelas_path, f'{os.path.splitext(gambar)[0]}_{sufiks_acak}{os.path.splitext(gambar)[1]}')
            
            # Salin gambar ke target path
            shutil.copy(source_path, target_path)

# Cetak ulang jumlah gambar per kelas setelah penyamaan
for kelas in kelas_aksara:
    kelas_path = os.path.join(data, kelas)
    jumlah_gambar = len(os.listdir(kelas_path))
    print(f'Aksara {kelas} jumlah: {jumlah_gambar}')

Aksara ba jumlah: 500
Aksara ca jumlah: 500
Aksara da jumlah: 500
Aksara dha jumlah: 500
Aksara ga jumlah: 500
Aksara ha jumlah: 500
Aksara ja jumlah: 500
Aksara ka jumlah: 500
Aksara la jumlah: 500
Aksara ma jumlah: 500
Aksara na jumlah: 500
Aksara nga jumlah: 500
Aksara nya jumlah: 500
Aksara pa jumlah: 500
Aksara ra jumlah: 500
Aksara sa jumlah: 500
Aksara ta jumlah: 500
Aksara tha jumlah: 500
Aksara wa jumlah: 500
Aksara ya jumlah: 500


### Split Dataset

In [11]:
import shutil
from sklearn.model_selection import train_test_split

# Path ke direktori utama yang berisi sub-direktori untuk masing-masing kelas
main_data_dir = output_dir

# Path ke direktori output untuk train dan test set
path_dir = '../data/data_preprocessing/data_split_v0.0/'

# Membuat sub-direktori train dan test
train_dir = os.path.join(path_dir, 'train')
test_dir = os.path.join(path_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Loop melalui masing-masing kelas (diasumsikan ada 20 kelas)
for class_name in os.listdir(main_data_dir):
    class_path = os.path.join(main_data_dir, class_name)
    
    # Mendapatkan list file untuk kelas tertentu
    files = os.listdir(class_path)
    
    # Membagi data menjadi train dan test set
    train_files, test_files = train_test_split(files, test_size=0.3, random_state=42)
    
    # Membuat sub-direktori untuk masing-masing kelas di train dan test set
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Menyalin file ke dalam masing-masing sub-direktori
    for file in train_files:
        shutil.copy(os.path.join(class_path, file), os.path.join(train_class_dir, file))
    
    for file in test_files:
        shutil.copy(os.path.join(class_path, file), os.path.join(test_class_dir, file))